In [1]:
!pip install plotly

     |████████████████████████████████| 11.5 MB 3.0 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=04b4a67f05bd66127ab6f31540b605ab4cb04557d1b09903ffaabb0e956dd85c
  Stored in directory: /root/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os, sys, time, json, joblib

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
sys.path.append("/data/ExternalTest/MAD/src/")
from constants import *
from metadata_utils import _find_files
from baseline_feats_utils import feat_type_feats_dct

In [4]:
feat_type_feats_dctMETADATA_DIR

{'user': ['num_interactions',
  'mean_price_interactions',
  'earliest_interaction_date',
  'min_num_interactions_per_pdt',
  'max_num_interactions_per_pdt',
  'mean_num_interactions_per_pdt',
  'min_num_interactions_per_ont',
  'max_num_interactions_per_ont',
  'mean_num_interactions_per_ont',
  'min_num_interactions_per_brand',
  'max_num_interactions_per_brand',
  'mean_num_interactions_per_brand'],
 'item': ['num_interactions',
  'earliest_interaction_date',
  'min_num_interactions_per_user',
  'max_num_interactions_per_user',
  'mean_num_interactions_per_user']}

In [5]:
def _prep_data(fn, group_col, feat_cols):
    df = pd.read_csv(fn, compression='gzip', sep='|')
    if isinstance(feat_cols, str):
        feat_cols = [feat_cols]
    needed_cols = [group_col] + feat_cols
    df = df[needed_cols]
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


def _concat_global_and_local_dfs(global_df, local_df, cols):
    
    df = pd.concat([global_df[cols], local_df[cols]], axis=0)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


def main(group_col, feat_cols, data_dir=INTERIM_DATA_DIR,
         end_token='.gz'):
    
    print('find the relevant files\n')
    files = [os.path.join(data_dir, x) for x in
             _find_files(data_dir, end_token)]
    
    print('Init global DF\n')
    global_df = pd.DataFrame(columns = [group_col] + feat_cols)
    
    print('Update global DF\n')
    for i, fn in enumerate(files):
        if i % 4 == 0:
            print('num files completed: %d' % (i))
            print('\n\n')
            
        print('prepare data - file %s' % (fn))
        local_df = _prep_data(fn, group_col, feat_cols)
        print('\n')
        
        print('concat global DF and local DF\n')
        global_df = _concat_global_and_local_dfs(global_df, local_df,
                                                 [group_col] + feat_cols)
        
        print('Shape after concatenation\n')
        print(global_df.shape)
        print('\n')
    
    print('\n\n\n')
    print('Final Shape of global DF: ', global_df.shape)
    print('Num unique group col: ', global_df[group_col].nunique())
    
    return global_df

In [7]:
%time df = main(USER_COL, ['uuid_num_interactions', 'uuid_mean_price_interactions'])

find the relevant files

Init global DF

Update global DF

num files completed: 0



prepare data - file /data/ExternalTest_Data/MAD/interim/0000_part_00.gz


concat global DF and local DF

Shape after concatenation

(921513, 3)
prepare data - file /data/ExternalTest_Data/MAD/interim/0005_part_05.gz


concat global DF and local DF

Shape after concatenation

(2578572, 3)
prepare data - file /data/ExternalTest_Data/MAD/interim/0001_part_07.gz


concat global DF and local DF

Shape after concatenation

(2829556, 3)
prepare data - file /data/ExternalTest_Data/MAD/interim/0003_part_07.gz


concat global DF and local DF

Shape after concatenation

(3036403, 3)
num files completed: 4



prepare data - file /data/ExternalTest_Data/MAD/interim/0004_part_01.gz


concat global DF and local DF

Shape after concatenation

(3520976, 3)
prepare data - file /data/ExternalTest_Data/MAD/interim/0004_part_05.gz


concat global DF and local DF

Shape after concatenation

(4221300, 3)
prepare data - file 

In [8]:
print(df.shape)
df.head()

(10130223, 3)


,uuid,uuid_num_interactions,uuid_mean_price_interactions
0,6301963,92.0,1783.997135
1,4316278,1264.0,2551.721414
2,1420289,455.0,1324.576459
3,4290616,1.0,44119.000000
4,7979798,206.0,5285.834471


In [9]:
df.memory_usage()

Index                                128
uuid                            81041784
uuid_num_interactions           81041784
uuid_mean_price_interactions    81041784
dtype: int64

In [10]:
df[['uuid_num_interactions', 'uuid_mean_price_interactions']].describe()

,uuid_num_interactions,uuid_mean_price_interactions
count,1.013022e+07,1.013022e+07
mean,1.711696e+01,1.729816e+04
std,7.052497e+01,3.049489e+04
min,1.000000e+00,-1.000000e+00
25%,2.000000e+00,2.595000e+03
50%,4.000000e+00,8.736500e+03
75%,1.000000e+01,2.161500e+04
max,4.208000e+03,5.999900e+06


In [11]:
df[['uuid_num_interactions', 'uuid_mean_price_interactions']].nunique()

uuid_num_interactions              3006
uuid_mean_price_interactions    2963906
dtype: int64

In [50]:
"""
trace = go.Histogram(x=df['uuid_num_interactions'],
                     name='num_user_interactions',
                     xbins=dict(start=0, end=151, size=20),
                     marker=dict(color='#db0000'))

layout = go.Layout(
    title='Distribution Of Number of User Interactions',
    xaxis=dict(title='Num user interactions'),
    yaxis=dict(title='Count'),
    bargap=0.2)

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
"""

"\ntrace = go.Histogram(x=df['uuid_num_interactions'],\n                     name='num_user_interactions',\n                     xbins=dict(start=0, end=151, size=20),\n                     marker=dict(color='#db0000'))\n\nlayout = go.Layout(\n    title='Distribution Of Number of User Interactions',\n    xaxis=dict(title='Num user interactions'),\n    yaxis=dict(title='Count'),\n    bargap=0.2)\n\nfig = go.Figure(data=[trace], layout=layout)\niplot(fig)\n"

In [20]:
# number of unique users in < 20 segment
mask = df['uuid_num_interactions'] < 20
print('num users: ', df['uuid'].nunique())
print('num users in < 20 segment: ', df.loc[mask, 'uuid'].nunique())
print('num users in >= 20 segment: ', df.loc[~mask, 'uuid'].nunique())

num users:  10130223
num users in < 20 segment:  8686053
num users in >= 20 segment:  1444170


In [46]:
mask = df['uuid_num_interactions'] < 20
df.loc[mask, 'uuid_num_interactions'].value_counts()

2.0     2650856
3.0     1304079
1.0      926509
4.0      902773
5.0      562435
6.0      420441
7.0      329595
8.0      266424
9.0      220288
10.0     185886
11.0     159410
12.0     138326
13.0     120345
14.0     107022
15.0      95630
16.0      85390
17.0      76874
18.0      69927
19.0      63843
Name: uuid_num_interactions, dtype: int64

In [47]:
mask1 = df['uuid_num_interactions'] <= 3
mask2 = df['uuid_num_interactions'] <= 19
print(df.loc[mask1, 'uuid'].nunique())
print(df.loc[~mask1&mask2, 'uuid'].nunique())
print(df.loc[~mask2, 'uuid'].nunique())

4881444
3804609
1444170


In [49]:
# create a new uuid-index map for >=20 segment


def _get_entity_index_map(entity_lst):

    user2idx = {user: i for i, user in enumerate(entity_lst)}
    idx2user = {i: user for i, user in enumerate(entity_lst)}

    return user2idx, idx2user

filter_dct = {
    'LE3': lambda data: data['uuid_num_interactions'] <= 3,
    '4-19': lambda data: ((data['uuid_num_interactions'] > 3) &
                          (data['uuid_num_interactions'] < 20)),
    'GE20': lambda data: data['uuid_num_interactions'] >= 20}

for k, func in filter_dct.items():
    print('key: ', k)
    print('\n')
    
    print('Apply filter\n')
    mask = func(df)
    uuids = df.loc[mask, 'uuid'].unique().tolist()
    uuids = [int(x) for x in uuids]
    print('num users: ', len(uuids))
    print('\n')
    
    print('get mapping\n')
    user2idx, idx2user = _get_entity_index_map(uuids)
    print(len(user2idx), '\t', len(idx2user))
    print('\n')
    
    print('save user2idx \n')
    out_fn = os.path.join(METADATA_DIR, 'user2idx_seg{}.json'.format(k))
    json.dump(user2idx, open(out_fn, 'w'))

    print('save idx2user \n')
    out_fn = os.path.join(METADATA_DIR, 'idx2user_seg{}.json'.format(k))
    json.dump(idx2user, open(out_fn, 'w'))
    
    print('\n\n\n')

key:  LE3


Apply filter

num users:  4881444


get mapping

4881444 	 4881444


save user2idx 

save idx2user 





key:  4-19


Apply filter

num users:  3804609


get mapping

3804609 	 3804609


save user2idx 

save idx2user 





key:  GE20


Apply filter

num users:  1444170


get mapping

1444170 	 1444170


save user2idx 

save idx2user 







In [28]:
list(user2idx.items())[:10]

[(6301963, 0),
 (4316278, 1),
 (1420289, 2),
 (7979798, 3),
 (9631081, 4),
 (5591379, 5),
 (1150508, 6),
 (8443048, 7),
 (6817371, 8),
 (5602548, 9)]

In [29]:
# save the mapping dictionaries
print('user2idx \n')
out_fn = os.path.join(METADATA_DIR, 'user2idx_segGE20.json')
json.dump(user2idx, open(out_fn, 'w'))

print('idx2user \n')
out_fn = os.path.join(METADATA_DIR, 'idx2user_segGE20.json')
json.dump(idx2user, open(out_fn, 'w'))

user2idx 

idx2user 



In [30]:
user2idx = json.load(open(os.path.join(METADATA_DIR, 'user2idx_segGE20.json')))

In [45]:
list(user2idx.items())[:10]

[('6301963', 0),
 ('4316278', 1),
 ('1420289', 2),
 ('7979798', 3),
 ('9631081', 4),
 ('5591379', 5),
 ('1150508', 6),
 ('8443048', 7),
 ('6817371', 8),
 ('5602548', 9)]